## 导入必要的包

In [1]:
from datasets import Dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, BitsAndBytesConfig, GenerationConfig
import torch
from typing import List, Tuple

In [2]:
print("torch version: ", torch.__version__)
print("transformers version: ", transformers.__version__)

torch version:  2.2.1+cu121
transformers version:  4.38.2


In [3]:
MAX_LENGTH = 512    # 分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
data_path = "../data/huanhuan.json"
model_dir = "../models/internlm2-chat-1_8b-sft"
work_dir = "../work_dirs/internlm2_lora_huanhuan"
system_prompt = "现在你要扮演皇帝身边的女人--甄嬛"

## 载入数据

In [4]:
# 使用datasets读取数据
df = pd.read_json(data_path)
df

,instruction,input,output
0,小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——,,嘘——都说许愿说破是不灵的。
1,这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱...,,你们俩话太多了，我该和温太医要一剂药，好好治治你们。
2,嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。,,出来走走，也是散心。
3,嬛妹妹，我虽是一介御医，俸禄微薄，可是我保证会一生一世对你好，疼爱你，保护你，永远事事以你为...,,实初哥哥这么说，就枉顾我们一直以来的兄妹情谊了，嬛儿没有哥哥，一直把你当作自己的亲哥哥一样看...
4,实初虽然唐突了妹妹，却是真心实意地希望妹妹不要去应选，这不仅仅是因为我心里一直把妹妹当成……...,,我们两家是世交，昔年恩义不过是父亲随手之劳，不必挂怀。
...,...,...,...
3724,你好,,皇上好，我是甄嬛，家父是大理寺少卿甄远道。
3725,你是谁？,,我是甄嬛，家父是大理寺少卿甄远道。
3726,你好,,皇上好，我是甄嬛，家父是大理寺少卿甄远道。
3727,你是谁？,,我是甄嬛，家父是大理寺少卿甄远道。


In [5]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3729
})

## 处理数据集

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True)
tokenizer

InternLM2Tokenizer(name_or_path='../models/internlm2-chat-1_8b-sft', vocab_size=92544, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|action_start|>', '<|action_end|>', '<|interpreter|>', '<|plugin|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	92538: AddedToken("<|plugin|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	92539: AddedToken("<|interpreter|>", rstrip=False, lstrip=False, single_word=False, 

In [7]:
def build_inputs(query: str, history: List[Tuple[str, str]] = [], meta_instruction="我是系统"):
    prompt = ""
    if meta_instruction:
        prompt += f"""<s>[UNUSED_TOKEN_146]system\n{meta_instruction}[UNUSED_TOKEN_145]\n"""
    else:
        prompt += "<s>"
    for record in history:
        prompt += f"""[UNUSED_TOKEN_146]user\n{record[0]}[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]assistant\n{record[1]}[UNUSED_TOKEN_145]\n"""
    prompt += f"""[UNUSED_TOKEN_146]user\n{query}[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]assistant\n"""
    return prompt

In [8]:
build_inputs('你哈')

'<s>[UNUSED_TOKEN_146]system\n我是系统[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]user\n你哈[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]assistant\n'

In [9]:
def process_func(example):

    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<s>[UNUSED_TOKEN_146]system{system_prompt}[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]user{example['instruction']}[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]assistant", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}[UNUSED_TOKEN_145]<s>", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [10]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [11]:
tokenizer.decode(tokenized_id[0]['input_ids'])

' <s><|im_start|> system现在你要扮演皇帝身边的女人--甄嬛<|im_end|> \n<|im_start|> user小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|> \n<|im_start|> assistant嘘——都说许愿说破是不灵的。<|im_end|><s></s>'

In [12]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

' 你们俩话太多了，我该和温太医要一剂药，好好治治你们。<|im_end|><s></s>'

## 创建模型

In [13]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # 是否在4位精度下加载模型。如果设置为True，则在4位精度下加载模型。
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float16,   # 4位精度计算的数据类型。这里设置为torch.float16，表示使用半精度浮点数。
    bnb_4bit_quant_type='nf4',              # 4位精度量化的类型。这里设置为"nf4"，表示使用nf4量化类型。 nf4: 4bit-NormalFloat
    bnb_4bit_use_double_quant=True,         # 是否使用双精度量化。如果设置为True，则使用双精度量化。
)
quantization_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map='auto',
    low_cpu_mem_usage=True,             # 是否使用低CPU内存，使用 device_map 参数必须为 True
    quantization_config=quantization_config,
)
model.enable_input_require_grads()      # 开启梯度检查点时，要执行该方法
model

InternLM2ForCausalLM(
  (model): InternLM2Model(
    (tok_embeddings): Embedding(92544, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x InternLM2DecoderLayer(
        (attention): InternLM2Attention(
          (wqkv): Linear4bit(in_features=2048, out_features=4096, bias=False)
          (wo): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): InternLM2DynamicNTKScalingRotaryEmbedding()
        )
        (feed_forward): InternLM2MLP(
          (w1): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (w3): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (w2): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (attention_norm): InternLM2RMSNorm()
        (ffn_norm): InternLM2RMSNorm()
      )
    )
    (norm): InternLM2RMSNorm()
  )
  (output): Linear(in_features=2048, out_features=92544, bias=False)
)

In [16]:
print(f"model.device: {model.device}, model.dtype: {model.dtype}")

model.device: cuda:0, model.dtype: torch.float16


## Lora

In [17]:
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training, load_peft_weights

In [18]:
# https://huggingface.co/docs/peft/developer_guides/quantization
model = prepare_model_for_kbit_training(model)
model

InternLM2ForCausalLM(
  (model): InternLM2Model(
    (tok_embeddings): Embedding(92544, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x InternLM2DecoderLayer(
        (attention): InternLM2Attention(
          (wqkv): Linear4bit(in_features=2048, out_features=4096, bias=False)
          (wo): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): InternLM2DynamicNTKScalingRotaryEmbedding()
        )
        (feed_forward): InternLM2MLP(
          (w1): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (w3): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (w2): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (attention_norm): InternLM2RMSNorm()
        (ffn_norm): InternLM2RMSNorm()
      )
    )
    (norm): InternLM2RMSNorm()
  )
  (output): Linear(in_features=2048, out_features=92544, bias=False)
)

In [19]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,   # 训练模式
    r=64,                   # Lora 秩
    target_modules=['wqkv', 'wo', 'w1', 'w2', 'w3'],
    lora_alpha=16,          # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,       # Dropout 比例
    bias='none'
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'w3', 'wo', 'wqkv', 'w2', 'w1'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [20]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='../models/internlm2-chat-1_8b-sft', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'w3', 'wo', 'wqkv', 'w2', 'w1'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [21]:
model.print_trainable_parameters()

trainable params: 7,864,320 || all params: 1,896,974,336 || trainable%: 0.41457176571944937


## 配置训练参数

In [22]:
args = TrainingArguments(
    output_dir=work_dir,
    optim="paged_adamw_32bit",
    learning_rate=1e-5,
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # 4*4=16
    logging_steps=10,
    num_train_epochs=3,
    save_steps=50,
    save_total_limit=3,
    save_on_each_node=True,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
)

In [23]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [24]:
trainer.train()

  0%|          | 0/699 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\miniconda3\envs\pytorch\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 5.7562, 'grad_norm': 7.630174160003662, 'learning_rate': 9.856938483547926e-06, 'epoch': 0.04}
{'loss': 5.2451, 'grad_norm': 10.935853004455566, 'learning_rate': 9.713876967095852e-06, 'epoch': 0.09}
{'loss': 4.7601, 'grad_norm': 4.187314510345459, 'learning_rate': 9.570815450643777e-06, 'epoch': 0.13}
{'loss': 4.4317, 'grad_norm': 6.8559794425964355, 'learning_rate': 9.427753934191704e-06, 'epoch': 0.17}
{'loss': 4.1082, 'grad_norm': 7.876904010772705, 'learning_rate': 9.284692417739628e-06, 'epoch': 0.21}


d:\miniconda3\envs\pytorch\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in ../models/internlm2-chat-1_8b-sft - will assume that the vocabulary was not modified.
  warnings.warn(
d:\miniconda3\envs\pytorch\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 3.7065, 'grad_norm': 4.046258926391602, 'learning_rate': 9.141630901287555e-06, 'epoch': 0.26}
{'loss': 3.5902, 'grad_norm': 4.043776988983154, 'learning_rate': 8.99856938483548e-06, 'epoch': 0.3}
{'loss': 3.5723, 'grad_norm': 5.262574195861816, 'learning_rate': 8.855507868383405e-06, 'epoch': 0.34}
{'loss': 3.5481, 'grad_norm': 3.2736563682556152, 'learning_rate': 8.712446351931331e-06, 'epoch': 0.39}
{'loss': 3.4757, 'grad_norm': 4.843493938446045, 'learning_rate': 8.569384835479256e-06, 'epoch': 0.43}


...

d:\miniconda3\envs\pytorch\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in ../models/internlm2-chat-1_8b-sft - will assume that the vocabulary was not modified.
  warnings.warn(
d:\miniconda3\envs\pytorch\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 3.2641, 'grad_norm': 3.7526755332946777, 'learning_rate': 1.2732474964234623e-06, 'epoch': 2.62}
{'loss': 3.1777, 'grad_norm': 4.780666351318359, 'learning_rate': 1.1301859799713878e-06, 'epoch': 2.66}
{'loss': 3.2875, 'grad_norm': 4.469741344451904, 'learning_rate': 9.871244635193133e-07, 'epoch': 2.7}
{'loss': 3.2268, 'grad_norm': 4.040610313415527, 'learning_rate': 8.44062947067239e-07, 'epoch': 2.74}
{'loss': 3.2484, 'grad_norm': 4.668625354766846, 'learning_rate': 7.010014306151645e-07, 'epoch': 2.79}


d:\miniconda3\envs\pytorch\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in ../models/internlm2-chat-1_8b-sft - will assume that the vocabulary was not modified.
  warnings.warn(
d:\miniconda3\envs\pytorch\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 3.3452, 'grad_norm': 5.963035583496094, 'learning_rate': 5.579399141630902e-07, 'epoch': 2.83}
{'loss': 3.311, 'grad_norm': 4.150168418884277, 'learning_rate': 4.148783977110158e-07, 'epoch': 2.87}
{'loss': 3.1485, 'grad_norm': 4.423460960388184, 'learning_rate': 2.718168812589414e-07, 'epoch': 2.92}
{'loss': 3.217, 'grad_norm': 4.400463104248047, 'learning_rate': 1.2875536480686695e-07, 'epoch': 2.96}
{'train_runtime': 2445.0788, 'train_samples_per_second': 4.575, 'train_steps_per_second': 0.286, 'train_loss': 3.4334965461654554, 'epoch': 3.0}


TrainOutput(global_step=699, training_loss=3.4334965461654554, metrics={'train_runtime': 2445.0788, 'train_samples_per_second': 4.575, 'train_steps_per_second': 0.286, 'train_loss': 3.4334965461654554, 'epoch': 3.0})